# Lesson 3: Enable Logging

### Import all needed packages

In [43]:
import boto3
import json
import os

bedrock = boto3.client('bedrock', region_name="us-west-2")

In [44]:
from helpers.CloudWatchHelper import CloudWatch_Helper
cloudwatch = CloudWatch_Helper()

In [45]:
log_group_name = '/my/amazon/bedrock/logs'

In [46]:
cloudwatch.create_log_group(log_group_name)

Log group '/my/amazon/bedrock/logs' already exists.


In [47]:
loggingConfig = {
    'cloudWatchConfig': {
        'logGroupName': log_group_name,
        'roleArn': os.environ['LOGGINGROLEARN'],
        'largeDataDeliveryS3Config': {
            'bucketName': os.environ['LOGGINGBUCKETNAME'],
            'keyPrefix': 'amazon_bedrock_large_data_delivery',
        }
    },
    's3Config': {
        'bucketName': os.environ['LOGGINGBUCKETNAME'],
        'keyPrefix': 'amazon_bedrock_logs',
    },
    'textDataDeliveryEnabled': True,
}

In [48]:
bedrock.put_model_invocation_logging_configuration(loggingConfig=loggingConfig)

{'ResponseMetadata': {'RequestId': 'edc4ec43-5d41-4a57-908a-b90d8688356c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 16 Apr 2024 19:05:15 GMT',
   'content-type': 'application/json',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'edc4ec43-5d41-4a57-908a-b90d8688356c'},
  'RetryAttempts': 0}}

In [49]:
bedrock.get_model_invocation_logging_configuration()

{'ResponseMetadata': {'RequestId': 'cad3f4c2-c86e-4e89-94cf-aa2a04f3084f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 16 Apr 2024 19:05:17 GMT',
   'content-type': 'application/json',
   'content-length': '572',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'cad3f4c2-c86e-4e89-94cf-aa2a04f3084f'},
  'RetryAttempts': 0},
 'loggingConfig': {'cloudWatchConfig': {'logGroupName': '/my/amazon/bedrock/logs',
   'roleArn': 'arn:aws:iam::358422811922:role/c99355a2566044l6454438t1w35842281192-LoggingIAMRole-z51O33QhufW9',
   'largeDataDeliveryS3Config': {'bucketName': 'c99355a2566044l6454438t1w358422811-loggings3bucket-5mvrv64gi0co',
    'keyPrefix': 'amazon_bedrock_large_data_delivery'}},
  's3Config': {'bucketName': 'c99355a2566044l6454438t1w358422811-loggings3bucket-5mvrv64gi0co',
   'keyPrefix': 'amazon_bedrock_logs'},
  'textDataDeliveryEnabled': True,
  'imageDataDeliveryEnabled': True,
  'embeddingDataDeliveryEnabled': True}}

In [50]:
bedrock_runtime = boto3.client('bedrock-runtime', region_name="us-west-2")

In [51]:
prompt = "Write an article about the fictional planet Foobar."

kwargs = {
    "modelId": "amazon.titan-text-express-v1",
    "contentType": "application/json",
    "accept": "*/*",
    "body": json.dumps(
        {
            "inputText": prompt,
            "textGenerationConfig": {
                "maxTokenCount": 512,
                "temperature": 0.7,
                "topP": 0.9
            }
        }
    )
}

response = bedrock_runtime.invoke_model(**kwargs)
response_body = json.loads(response.get('body').read())

generation = response_body['results'][0]['outputText']

print(generation)


Foobar is a fictional planet in the Star Wars universe. It is first mentioned in the 1977 film Star Wars: Episode IV - A New Hope, where it is described as a remote and isolated planet located in the Outer Rim. The planet is home to a species of sentient, slug-like creatures known as the Jawas, who are known for their scavenging and trading practices.

Despite its remote location, Foobar has played a significant role in the Star Wars story. The planet was the site of a major battle between the Rebel Alliance and the Empire in the film, where the Rebels were able to destroy the Empire's new Death Star superweapon. The planet was also the location of the first meeting between Han Solo and Princess Leia, which set the stage for the rest of the franchise.

In the expanded universe of Star Wars, Foobar has been further explored in various books, comics, and other media. The planet has been home to a number of important characters, including Jawa trader Unkar Plutt, who was a key ally of th

In [55]:
cloudwatch.print_recent_logs(log_group_name)

Permissions are correctly set for Amazon Bedrock logs.
-------------------------

{
    "schemaType": "ModelInvocationLog",
    "schemaVersion": "1.0",
    "timestamp": "2024-04-16T19:05:22Z",
    "accountId": "358422811922",
    "identity": {
        "arn": "arn:aws:sts::358422811922:assumed-role/voclabs/user3224868=0"
    },
    "region": "us-west-2",
    "requestId": "bb2b0bbe-af3f-42b6-be43-8dc69a17564b",
    "operation": "InvokeModel",
    "modelId": "amazon.titan-text-express-v1",
    "input": {
        "inputContentType": "application/json",
        "inputBodyJson": {
            "inputText": "Write an article about the fictional planet Foobar.",
            "textGenerationConfig": {
                "maxTokenCount": 512,
                "temperature": 0.7,
                "topP": 0.9
            }
        },
        "inputTokenCount": 10
    },
    "output": {
        "outputContentType": "application/json",
        "outputBodyJson": {
            "inputTextTokenCount": 10,
    

To review the logs within the AWS console, please use the following link to reference the steps outlined in the video:

In [53]:
from IPython.display import HTML
aws_url = os.environ['AWS_CONSOLE_URL']

In [ ]:
HTML(f'<a href="{aws_url}" target="_blank">GO TO AWS CONSOLE</a>')
